# Подготовка данных для обучения сети

Заметки по bash:
* `rm -rf './model'` - удалить папку
* `rm ./data/train_Sample150k_Vac134k_Seq256.tfrecords` - удалить файл
* `mv что_переместить куда_переместить`
* `!ls -all ./data` - показать файлы и дополнительную информацию

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.utils import class_weight
from pathlib import Path
import os

In [2]:
HP_DATASET = {'dataset_size_resampled' : 500_000,
              'max_features'           : 50_000,
              'max_sequence_length'    : 128,
              'batch_size'             : 2048,
             }

In [3]:
path_data = Path('./data/')
path_models = Path('./models/')

filename_train_cleaned                         = f"train_cleaned.tfrecords"
filename_train_clean_encoded                   = f"train_clean_encoded.tfrecords"
filename_train_clean_encode_resampled          = f"train_clean_encode_resampled.tfrecords"


filename_test_cleaned                          = f"test_cleaned.tfrecords"
filename_test_clean_encoded                    = f"test_clean_encoded.tfrecords"

filename_model_int_vect                        = f"int_vect_layer_75k_128e"

In [4]:
if os.path.exists(path_data):
    pass
else:
    os.makedirs(path_data)
    print('Folder `data` was create')

    
if os.path.exists(path_models):
    pass
else:
    os.makedirs(path_models)
    print('Folder `models` was create')

Folder `data` was create
Folder `models` was create


# 1 Загрузка txt в Dataset

In [5]:
def load_data(path_filename):
  # Создание текстового dataset считывает указанный файл
  dataset = tf.data.TextLineDataset([path_filename])
  # Разделение записей (строка) на элементы по разделителю
  dataset = dataset.map(lambda x: tf.strings.split(x, ' ::: '))
  # Разделение на отдельных "записей" - изменение размерности (4, ) -> (4, 1)
  dataset = dataset.map(lambda x: tf.stack(tf.split(x, num_or_size_splits=4)))
  # фильтр полей - остаются только жанр и описание
  # можно построить словарь и в дальнейшем обращаться к нужной части
  dataset = dataset.map(lambda x: {'label': x[2][0], 'text': x[3][0]})
  return dataset

In [6]:
# Загрузка и разделение на описание и лейблы
path_train_data = '../input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt'

train_ds = load_data(path_train_data)

for i in train_ds.take(1):
    print(i)

2022-08-05 11:41:27.582951: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


{'label': <tf.Tensor: shape=(), dtype=string, numpy=b'drama'>, 'text': <tf.Tensor: shape=(), dtype=string, numpy=b'Listening in to a conversation between his doctor and parents, 10-year-old Oscar learns what nobody has the courage to tell him. He only has a few weeks to live. Furious, he refuses to speak to anyone except straight-talking Rose, the lady in pink he meets on the hospital stairs. As Christmas approaches, Rose uses her fantastical experiences as a professional wrestler, her imagination, wit and charm to allow Oscar to live life and love to the full, in the company of his friends Pop Corn, Einstein, Bacon and childhood sweetheart Peggy Blue.'>}


2022-08-05 11:41:27.824742: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


# 2 Кодирование жанров

In [7]:
def encode_label(dataset):
  # Создание словаря \ таблицы для кодирования классов
  # Для этого используется статичный "словарь" - таблица.
  init = tf.lookup.KeyValueTensorInitializer(
      keys=tf.constant([b'drama', b'thriller', b'adult', b'documentary', b'comedy',
                        b'crime', b'reality-tv', b'horror', b'sport', b'animation',
                        b'action', b'fantasy', b'short', b'sci-fi', b'music', b'adventure',
                        b'talk-show', b'western', b'family', b'mystery', b'history',
                        b'news', b'biography', b'romance', b'game-show', b'musical', b'war']),
      values=tf.constant([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 
                          17, 18, 19, 20, 21, 22, 23, 24, 25, 26, ], dtype=tf.int64))

  table = tf.lookup.StaticVocabularyTable(init, num_oov_buckets=5)

  # Заменяем жанры на цифры - применяем полученную таблицу к жанрам.
  # Сейчас записи сохранены в виде словаря
  
  return (dataset['text'], table[dataset['label']])

In [8]:
# Кодирование лейблов (жанров)
train_ds = train_ds.map(encode_label)
for i in train_ds.take(1):
    print(i)

(<tf.Tensor: shape=(), dtype=string, numpy=b'Listening in to a conversation between his doctor and parents, 10-year-old Oscar learns what nobody has the courage to tell him. He only has a few weeks to live. Furious, he refuses to speak to anyone except straight-talking Rose, the lady in pink he meets on the hospital stairs. As Christmas approaches, Rose uses her fantastical experiences as a professional wrestler, her imagination, wit and charm to allow Oscar to live life and love to the full, in the company of his friends Pop Corn, Einstein, Bacon and childhood sweetheart Peggy Blue.'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


# 3 Очистка текста от stop слов

In [9]:
import nltk
from nltk.corpus import stopwords

stopwords_eng = stopwords.words('english')

def clean_data(dataset, stop_words):
    punctuation = """[!"#$%&'()*+,./:;<=>?@[\]^_`{|}~]"""
    doc, label = dataset
    # приведение к нижнему регистру
    doc = tf.strings.lower(doc)
    # заменяем `-` для отдельных слов, чтобы не сливалось 10year
    doc = tf.strings.regex_replace(doc, '-', ' ', replace_global=True, name=None)
    # замена всей пунктуации
    doc = tf.strings.regex_replace(doc, punctuation, '', replace_global=True, name=None)
    for word in stop_words:
        # замена стоп слов, слово целиком
        doc = tf.strings.regex_replace(doc, f'\\b{word}\\b', ' ', replace_global=True, name=None)
        # лишние пробелы после предыдущей замены
        doc = tf.strings.regex_replace(doc, f' +', ' ', replace_global=True, name=None)
        # пробелы вначале и конце тензора
        doc = tf.strings.strip(doc, name=None)
    return doc, label

In [10]:
train_ds = train_ds.map(lambda *x: clean_data(x, stopwords_eng))

for i in train_ds.take(1):
    print(i)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Inconsistent ASTs detected. This is a bug. Cause: 
inconsistent values for field value: \b and Diff:
*** Original nodes

--- Reparsed nodes

***************

*** 521,541 ****

  | | | | | | | | | | | names=[
  | | | | | | | | | | | | "doc"
  | | | | | | | | | | | ]
  | | | | | | | | | | Assign:
  | | | | | | | | | | | targets=[
  | | | | | | | | | | | | Name:
  | | | | | | | | | | | | | id=u"word"
! | | | | | | | | | | | | | ctx=Store:
  | | | | | | | | | | | | | annotation=None
  | | | | | | | | | | | | | type_comment=None
  | | | | | | | | | | | ]
  | | | | | | | | | | | value=Name:
  | | | | | | | | | | | | id=u"itr"
! | | | | | | | | | | | | ctx=Load:
  | | | | | | | | | | | | annotation=None
  | | | | | | | | | | | | type_comment=None
  | | | | | | | | | | Assign:
  | | | | | | | | | | | targets=[
  | | | | | | | | 

# 4 Сохранение набора данных в TFRecords (после очистки)

Создание набора данных и запись в TFRecords занимает много времени, операция проведена один, раз после используется подготовленный файл.

## Функции сохранения

In [11]:
def int64_feature(label):
  label = tf.train.Feature(int64_list=tf.train.Int64List(value=[label])) 
  return label


def bytes_feature(description):
  if isinstance(description, type(tf.constant(0))):
    description = description.numpy()
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[description]))


def float_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def create_record(description, label):
    feature = {
        'description': description,
        'label': label
      }
    proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return proto.SerializeToString()

## Функции загрузки

In [12]:
feature_description = {
    'description': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'label': tf.io.FixedLenFeature([], tf.int64, default_value=0),
}

def parse_function(example_proto):
  parsed = tf.io.parse_single_example(example_proto, feature_description)
  return (parsed['description'], parsed['label']) 

## Сохранение очищенных данных

In [13]:
path = str(path_data / filename_train_cleaned)

with tf.io.TFRecordWriter(path) as writer:
    """dataset: 
       sample0 - text_description; 
       sample1 - label
    """
    for sample in train_ds:
        label = int64_feature(sample[1])
        desc = bytes_feature(sample[0])
        pr = create_record(desc, label)
        writer.write(pr)
    print(f'dataset сохранён: {path}')

dataset сохранён: data/train_cleaned.tfrecords


## Проверка загрузки

In [14]:
path = str(path_data / filename_train_cleaned)

train_ds = tf.data.TFRecordDataset(filenames = [path])
train_ds = train_ds.map(parse_function)

for i in train_ds.take(1):
    print(i)

d_len = []
for i in train_ds:
    d_len.append(i)

print('Набор данных количество записей:', len(d_len))

words_amount = []
for i in train_ds:
    str_desc = i[0].numpy().decode()
    words_amount.extend(str_desc.split(' '))

print('Количество слов (для словаря):', len(set(words_amount)))

(<tf.Tensor: shape=(), dtype=string, numpy=b'listening conversation doctor parents 10 year old oscar learns nobody courage tell weeks live furious refuses speak anyone except straight talking rose lady pink meets hospital stairs christmas approaches rose uses fantastical experiences professional wrestler imagination wit charm allow oscar live life love full company friends pop corn einstein bacon childhood sweetheart peggy blue'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
Набор данных количество записей: 54214
Количество слов (для словаря): 134252


**Всего 134252 уникальных токена в наборе**

# 5 Кодирование данных для обучения модели

Векторизация описания фильма:
* Создаётся словарь заданной длины.
* Каждому слову присваивается номер.
* Слова в описании заменяются номерами.
* Если длина описания фильма больше указанной, то описание обрезается.
* Если меньше то дополняется нулями.

In [15]:
def get_description_encoder(dataset, MAX_FEATURES, MAX_SEQUENCE_LENGTH):
  # Слой препроцесинга. Инициализация
  int_vectorize_layer = tf.keras.layers.TextVectorization(
      max_tokens=MAX_FEATURES,
      standardize = 'lower_and_strip_punctuation',
      output_mode='int',
      output_sequence_length=MAX_SEQUENCE_LENGTH)

  # Оставляем только описание фильма (без меток жанров)
  dataset = dataset.map(lambda f, l: f)

  # "обучение" препроцессинга для кодирования
  int_vectorize_layer.adapt(dataset)
  return int_vectorize_layer

In [16]:
# ⚠️Загружается очищенный набор, БЕЗ балансировки
path = str(path_data / filename_train_cleaned)

train_ds = tf.data.TFRecordDataset(filenames = [path])
train_ds = train_ds.map(parse_function)

# Создание кодировщика текста
int_vectorize_layer = get_description_encoder(train_ds, 
                                              MAX_FEATURES=HP_DATASET['max_features'], 
                                              MAX_SEQUENCE_LENGTH=HP_DATASET['max_sequence_length'])

In [17]:
# Пример кодирования изначального текста
path_train_data = '../input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt'
example_ds = load_data(path_train_data)

# 1 - неизвестные слова, 0 - padding
for i in example_ds.take(1):
  print('Текстовое описание:')
  print(i)
  print('Кодирование:')
  print(int_vectorize_layer(i['text']))
  print('Декодирование первого слова:')
  print(int_vectorize_layer.get_vocabulary()[7001])

Текстовое описание:
{'label': <tf.Tensor: shape=(), dtype=string, numpy=b'drama'>, 'text': <tf.Tensor: shape=(), dtype=string, numpy=b'Listening in to a conversation between his doctor and parents, 10-year-old Oscar learns what nobody has the courage to tell him. He only has a few weeks to live. Furious, he refuses to speak to anyone except straight-talking Rose, the lady in pink he meets on the hospital stairs. As Christmas approaches, Rose uses her fantastical experiences as a professional wrestler, her imagination, wit and charm to allow Oscar to live life and love to the full, in the company of his friends Pop Corn, Einstein, Bacon and childhood sweetheart Peggy Blue.'>}
Кодирование:
tf.Tensor(
[ 5135     1     1     1  1916     1     1   423     1   123     1  2550
   337     1  2186     1     1  1339     1   351     1     1     1     1
     1     1  1044     1    51  4331     1   840     1  1344     1   755
  1796     1  1307     1   682     1  4897     1   106     1     1   463


## Кодирование и сохранение очищенных данных

In [18]:
def encode_description(dataset, int_vectorize_layer):
  # Кодируем тренировочный dataset (текст)
  def int_vectorize_text(*ds):
    return int_vectorize_layer(ds[0]), ds[1]
  
  dataset = dataset.map(int_vectorize_text)
  return dataset

In [19]:
path = str(path_data / filename_train_cleaned)

train_ds = tf.data.TFRecordDataset(filenames = [path])
train_ds = train_ds.map(parse_function)

train_ds_encoded = encode_description(train_ds, int_vectorize_layer)

In [20]:
# Сохранение на диск
path = str(path_data / filename_train_clean_encoded)

with tf.io.TFRecordWriter(path) as writer:
    for sample in train_ds_encoded:
        label = int64_feature(sample[1])
        # Для преобразования в скаляр, array напрямую нельзя кодировать
        desc_serialized = tf.io.serialize_tensor(sample[0])
        desc = bytes_feature(desc_serialized)
        pr = create_record(desc, label)
        writer.write(pr)

In [21]:
# Проверка 
path = str(path_data / filename_train_clean_encoded)

train_ds = tf.data.TFRecordDataset(filenames = [path])
train_ds = train_ds.map(parse_function)

# Десcериализация строки в array
train_ds = train_ds.map(lambda x,y: (tf.io.parse_tensor(x, out_type=tf.int64), y))

for i in train_ds.take(1):
    print(i)

(<tf.Tensor: shape=(128,), dtype=int64, numpy=
array([ 5135,  1916,   423,   123,   677,    28,    16,  2550,   337,
        2186,  1339,   351,  1044,    51,  4331,   840,  1344,   755,
        1796,  1485,  1322,  1307,   682,  4897,   106,   463,  8694,
         741,  2726,  1307,   768, 10628,   484,   608,  9057,  1810,
        4918,  3608,  1792,  2550,    51,     2,    10,   275,   240,
          30,  1073, 12241, 15162, 12587,   376,  2971,  7744,  1173,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,  

## Сохранение кодировщика в `save model`

In [22]:
# Создание модели из одного слоя (кодировщик)
vectorize_layer_model = tf.keras.models.Sequential()
vectorize_layer_model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
vectorize_layer_model.add(int_vectorize_layer)
vectorize_layer_model.summary()

# Сохранение модели
path = str(path_models / filename_model_int_vect)
vectorize_layer_model.save(path, save_format="tf")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 128)               0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


2022-08-05 11:45:03.829237: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [23]:
# Загрузка сохранённой модели (кодировщик)
path = str(path_models / filename_model_int_vect)

loaded_vectorize_layer_model = tf.keras.models.load_model(path)
loaded_vectorize_layer = loaded_vectorize_layer_model.layers[0]

In [24]:
# Проверка загруженной модели (кодировщик)
# ⚠️ В текущей версии tf в kaggle ошибка, при загрузке не срабатывает конфиг, 
# тензор меньшего размера, отсекаются нули
# Подаём очищенный, но ещё не закодированный набор
path = str(path_data / filename_train_cleaned)

train_ds = tf.data.TFRecordDataset(filenames = [path])
train_ds = train_ds.map(parse_function)

# Кодирование набора данных
train_ds = encode_description(train_ds, loaded_vectorize_layer)

for i in train_ds.take(1):
    print(i)

(<tf.Tensor: shape=(54,), dtype=int64, numpy=
array([ 5135,  1916,   423,   123,   677,    28,    16,  2550,   337,
        2186,  1339,   351,  1044,    51,  4331,   840,  1344,   755,
        1796,  1485,  1322,  1307,   682,  4897,   106,   463,  8694,
         741,  2726,  1307,   768, 10628,   484,   608,  9057,  1810,
        4918,  3608,  1792,  2550,    51,     2,    10,   275,   240,
          30,  1073, 12241, 15162, 12587,   376,  2971,  7744,  1173])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


# 6 Балансировка классов с повторениями - `tf.data.Dataset.sample_from_datasets`


* Если набор данных сбалансированный, то обучение более плавное т.к. объекты повторяются в разных выборках, но вес их одинаковый по сравнению с `class_weight`
* т.е. проходов больше, но график обучения более плавный из-за того, что данные размазаны.
* если подавать в `model.fit()` из-за большого количества классов и большого количества преобразований, фильтрации, обучение будет очень долгим.
* по этому преобразуем в готовый набор данных и сохраним в `TFRecords`.

*Примечания:*
* если подавать напрямую, то нужно указать сколько шагов в эпохе т.к. функция балансировки выдаёт бесконечное количество образцов.

```python
# Параметр нужен т.к. бесконечные повторения элементов - эпоха выбрать все элементя один раз
steps_per_epoch = int(np.ceil(54_214 / conf.BATCH_SIZE))
```

In [25]:
def generate_infinity_dataset(dataset, class_id):
  # Фильтрует набор, набирает в буфер 10k элементов, возвращает бесконечное количество элементов.

  dataset_filterd = dataset.filter(lambda x, y: y == class_id)
  # dataset.shuffle(buffer_size=3) - создаётся буфер размера N (например = 3) в который помещаются 3 рандомных элемента, 
  # постепенно один за другим выбираются из него и заменяются теми, что не вошли в буфер
  # чем больше тем лучше
  dataset_filterd = dataset_filterd.shuffle(10_000)
  # dataset.repeat(3) - сколько раз реинацилизировать полный dataset - сколько раз подавать в эпоху dataset.
  # ⚠️ .reapeat(-1) или repeat() - бесконечное повторение
  dataset_filterd = dataset_filterd.repeat(-1)
  return dataset_filterd

In [26]:
path = str(path_data / filename_train_clean_encoded)

train_ds = tf.data.TFRecordDataset(filenames = [path])
train_ds = train_ds.map(parse_function)

# Десcериализация строки в array
train_ds = train_ds.map(lambda x,y: (tf.io.parse_tensor(x, out_type=tf.int64), y))


filtered_dataset = []
for class_id in range(27):
    filtered_dataset.append(generate_infinity_dataset(train_ds, class_id))

print('Количество поднаборов по жанрам:', len(filtered_dataset))

for i in filtered_dataset[0].take(1):
    print(i)

Количество поднаборов по жанрам: 27
(<tf.Tensor: shape=(128,), dtype=int64, numpy=
array([    8,    10,  2647,  2190,   994,     1,     1,   105,   254,
        2932,     1, 11313,    38,   404,   250,  8975,    39,  5148,
         661,   522,   493,  4353,   188,     4,  1065,  1761,   635,
          25,   920,    82,   943,  1408,   111,    20,     2,   422,
         379,    89,  1055,   832,  1746,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
       

In [27]:
# Балансировка набора данных
weight_list = [1*0.037 for x in range(0,27)]
# Формируется сбалансированный набор данных из других наборов
train_ds_resampled = tf.data.experimental.sample_from_datasets(filtered_dataset, weights=weight_list)
# example_ds_resampled = tf.data.Dataset.sample_from_datasets(filtered_dataset, weights=weight_list)

train_ds_resampled = train_ds_resampled.prefetch(10_000)

In [28]:
path = str(path_data / filename_train_clean_encode_resampled)

with tf.io.TFRecordWriter(path) as writer:
    for sample in train_ds_resampled.take(HP_DATASET['dataset_size_resampled']):    
        label = int64_feature(sample[1])
        # Для преобразования в скаляр, array напрямую нельзя кодировать
        desc_serialized = tf.io.serialize_tensor(sample[0])
        desc = bytes_feature(desc_serialized)
        pr = create_record(desc, label)
        writer.write(pr)

2022-08-05 11:48:39.657830: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 580 of 10000
2022-08-05 11:48:39.739988: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


In [29]:
# Проверка записанных данных
path = str(path_data / filename_train_clean_encode_resampled)
train_ds_resampled = tf.data.TFRecordDataset(filenames = [path])
train_ds_resampled = train_ds_resampled.map(parse_function)
train_ds_resampled = train_ds_resampled.map(lambda x,y: (tf.io.parse_tensor(x, out_type=tf.int64), y))

len_tfr = []
for i in train_ds_resampled:
    len_tfr.append(i)
print('Набор объёмом:', len(len_tfr))

for i in train_ds_resampled.take(1):
    print(i)

Набор объёмом: 500000
(<tf.Tensor: shape=(128,), dtype=int64, numpy=
array([12493,   146,  1056, 15844,  5633,    17,   600,   218,  1899,
        1905,   540,   498, 12493,  1140,  1453,   180, 30678,  2295,
       12658,     1, 12649, 12983,  2440,     1,  2960,     1,    67,
        9043, 48673,   702,     1,  3143, 23009,  7891, 10238,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0, 

# 7 Балансировка классов `class_weight`

Жанры не сбалансированы в наборе данных. Вместо балансировки `dataset`, можно задавать "важность(вес)" `class_weight` для классов. Меньше объектов какого-либо класса, важность **класса** выше.

* ❗`class_weight` - Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to **"pay more attention" to samples from an under-represented class**. 
* ⚠️Параметр задаётся в `model.fit()`.


Также есть другой параметр: `Sample weights` - увеличивает роль отдельного объекта (наблюдения).

* ❗`Sample weights` are used to increase the importance of a single data-point (let's say, some of your data is more trustworthy, then they receive a higher weight).

So: `The sample weights` exist to change the importance of data-points whereas the `class weights` change the weights to correct class imbalance.

[Дополнительная информация - Quora](https://www.quora.com/What-is-the-difference-between-class_weight-and-sample_weight-in-keras)

In [30]:
def calc_class_weight(dataset):
  # Расчёт весов для class_weight
  class_list = []
  for i in dataset:
      class_list.append(i[1].numpy())

  amount = len(class_list)
  print(f'Всего записей: {amount}')

  unique_class = set(class_list)

  class_pct = {}

  for i in unique_class:
      class_pct[i] = round(class_list.count(i) / amount, 7)

  labels_unique = list(set(class_list))

  class_weight_ = class_weight.compute_class_weight(
                  class_weight = 'balanced',
                  classes = labels_unique, 
                  y = np.array(class_list))

  weighted_class = {}

  for class_name, weight in zip(unique_class, class_weight_):
      weighted_class[class_name] = weight

  return weighted_class

In [31]:
# Как сбалансированы классы в сгенерированном dataset, который подаётся в fit()
# Проверка записанных данных
path = str(path_data / filename_train_clean_encode_resampled)
train_ds_resampled = tf.data.TFRecordDataset(filenames = [path])
train_ds_resampled = train_ds_resampled.map(parse_function)
train_ds_resampled = train_ds_resampled.map(lambda x,y: (tf.io.parse_tensor(x, out_type=tf.int64), y))


weighted_class = calc_class_weight(train_ds_resampled)

avg = np.mean(list(weighted_class.values()))

print(f'Балансировка классов - {avg}')

weighted_class

Всего записей: 500000
Балансировка классов - 1.0000488753825723


{0: 1.0096787807926786,
 1: 0.9939626707379378,
 2: 1.003061343219506,
 3: 1.003224363103013,
 4: 1.0079750989831546,
 5: 0.9992186110461619,
 6: 0.9938559823173143,
 7: 1.0073719479148409,
 8: 0.9943362606592847,
 9: 0.9980876640357076,
 10: 1.0076460179844662,
 11: 1.0078105316200554,
 12: 1.0003521239476296,
 13: 0.9955657501488371,
 14: 1.0042580541495942,
 15: 0.9935893614399892,
 16: 0.9937493167973447,
 17: 1.0074267500010075,
 18: 0.992311569955981,
 19: 0.9846609516945031,
 20: 0.9993264539700242,
 21: 0.9970129492041843,
 22: 0.9969592742136484,
 23: 1.0175010175010175,
 24: 0.9923647456469921,
 25: 1.0022470378588797,
 26: 0.9977650063856961}

In [32]:
# Как сбалансированы классы в начальном наборе данных
path = str(path_data / filename_train_clean_encoded)
train_ds = tf.data.TFRecordDataset(filenames = [path])
train_ds = train_ds.map(parse_function)
train_ds = train_ds.map(lambda x,y: (tf.io.parse_tensor(x, out_type=tf.int64), y))


weighted_class = calc_class_weight(train_ds)

avg = np.mean(list(weighted_class.values()))

print(f'Балансировка классов - {avg}')
print(weighted_class)

weights = {}
for key, value in weighted_class.items():
    weights[key] = round(value, 4)

Всего записей: 54214
Балансировка классов - 4.308273963785575
{0: 0.1475006189617223, 1: 1.2620527504248436, 2: 3.403264281230383, 3: 0.15332360460643907, 4: 0.26962883388289594, 5: 3.976090942427576, 6: 2.271409418468242, 7: 0.9110371714727432, 8: 4.647976680384088, 9: 4.031979770935594, 10: 1.5269398676242782, 11: 6.216488934755189, 12: 0.3958064115761731, 13: 3.103440380101895, 14: 2.74682069210113, 15: 2.590872162485066, 16: 5.135360424362982, 17: 1.945664656904967, 18: 2.561130007558579, 19: 6.294438639266225, 20: 8.263069654016157, 21: 11.093513402905668, 22: 7.577078965758211, 23: 2.987985008818342, 24: 10.350133638793432, 25: 7.248830057494318, 26: 15.211560044893378}


In [33]:
weights = {0: 0.1475, 1: 1.2621, 2: 3.4033, 3: 0.1533, 4: 0.2696, 5: 3.9761,
           6: 2.2714, 7: 0.911, 8: 4.648, 9: 4.032, 10: 1.5269, 11: 6.2165, 
           12: 0.3958, 13: 3.1034, 14: 2.7468, 15: 2.5909, 16: 5.1354, 17: 1.9457,
           18: 2.5611, 19: 6.2944, 20: 8.2631, 21: 11.0935, 22: 7.5771, 23: 2.988,
           24: 10.3501, 25: 7.2488, 26: 15.2116}

# 8 Кодирование тестового набора данных

## Загрузка и очистка данных -> `test_cleaned.tfrecords`

In [34]:
# Загрузка и разделение на описание и лейблы
path_test_data = '../input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt'

test_ds = load_data(path_test_data)
test_ds = test_ds.map(encode_label)

# Очистка данных от лишнего
test_ds = test_ds.map(lambda *x: clean_data(x, stopwords_eng))

# Сохранение очищенного набора
path = str(path_data / filename_test_cleaned)

with tf.io.TFRecordWriter(path) as writer:
  for sample in test_ds:
      label = int64_feature(sample[1])
      desc = bytes_feature(sample[0])
      pr = create_record(desc, label)
      writer.write(pr)

In [35]:
for i in test_ds.take(1):
    print(i)

(<tf.Tensor: shape=(), dtype=string, numpy=b'lr brane loves life car apartment job especially girlfriend vespa one day showering vespa runs shampoo lr runs across street convenience store buy quick trip minutes returns vespa gone every trace existence wiped lrs life becomes tortured existence one strange event another occurs confirm mind conspiracy working finding vespa'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)


## Кодирование набора данных -> `test_clean_encoded.tfrecords`

In [36]:
path = str(path_data / filename_test_cleaned)
test_ds = tf.data.TFRecordDataset(filenames = [path])
test_ds = test_ds.map(parse_function)

# Кодирование тестового набора
# ⚠️не проверял, можно ли кодировать загруженным слоем (теряет нули)
test_ds_clean_encoded = encode_description(test_ds, int_vectorize_layer)

In [37]:
# Сохранение на диск
path = str(path_data / filename_test_clean_encoded)

with tf.io.TFRecordWriter(path) as writer:
    for sample in test_ds_clean_encoded:
        label = int64_feature(sample[1])
        # Для преобразования в скаляр, array напрямую нельзя кодировать
        desc_serialized = tf.io.serialize_tensor(sample[0])
        desc = bytes_feature(desc_serialized)
        pr = create_record(desc, label)
        writer.write(pr)

In [38]:
# Проверка 
path = str(path_data / filename_test_clean_encoded)

test_ds = tf.data.TFRecordDataset(filenames = [path])
test_ds = test_ds.map(parse_function)
test_ds = test_ds.map(lambda x,y: (tf.io.parse_tensor(x, out_type=tf.int64), y))

for i in test_ds.take(1):
    print(i)

(<tf.Tensor: shape=(128,), dtype=int64, numpy=
array([38379,     1,   546,     2,   223,   561,   109,   782,   262,
       24791,     3,    20, 26998, 24791,   480, 27019, 38379,   480,
         204,   304,  7051,  1009,  1127,  2099,   296,  1324,   236,
       24791,   867,   111,  3499,   803,  8292,     1,     2,    72,
        3326,   803,     3,   380,   576,    99,  4446, 12863,   370,
        2684,   149,   588, 24791,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,  

In [39]:
!ls -all /kaggle/working

total 104
drwxr-xr-x 4 root root  4096 Aug  5 11:41 .
drwxr-xr-x 6 root root  4096 Aug  5 11:41 ..
---------- 1 root root 89547 Aug  5 13:02 __notebook__.ipynb
drwxr-xr-x 2 root root  4096 Aug  5 13:01 data
drwxr-xr-x 3 root root  4096 Aug  5 11:45 models


In [40]:
!ls -all /kaggle/working/data

total 701528
drwxr-xr-x 2 root root      4096 Aug  5 13:01 .
drwxr-xr-x 4 root root      4096 Aug  5 11:41 ..
-rw-r--r-- 1 root root  59403200 Aug  5 13:02 test_clean_encoded.tfrecords
-rw-r--r-- 1 root root  25756938 Aug  5 13:01 test_cleaned.tfrecords
-rw-r--r-- 1 root root 548000000 Aug  5 12:55 train_clean_encode_resampled.tfrecords
-rw-r--r-- 1 root root  59418544 Aug  5 11:45 train_clean_encoded.tfrecords
-rw-r--r-- 1 root root  25763511 Aug  5 11:43 train_cleaned.tfrecords


# 9 Архивирование подготовленных данных

In [41]:
# import shutil
# shutil.make_archive(base_name='data', format='zip', base_dir='/kaggle/working/data')
# shutil.make_archive(base_name='models', format='zip', base_dir='/kaggle/working/models')